In [1]:
import geopandas as gpd
import plotly
import matplotlib
import pandas as pd

In [2]:
tazs = gpd.read_file('G:\Data_Resources\modx\canonical_TAZ_shapefile\candidate_CTPS_TAZ_STATEWIDE_2019.shp')
bike_nodes = gpd.read_file('E:\Shared drives\TMD_TSA\Data\\Non_Motorized_Networks\Bike\Shapefiles_to_DBD\\nodes.shp')
ped_nodes = gpd.read_file('E:\Shared drives\TMD_TSA\Data\\Non_Motorized_Networks\Ped\Shapefiles_to_DBD\\nodes.shp')

In [3]:
#make sure everything is in the projected crs
tazs = tazs.to_crs(26986)
bike_nodes = bike_nodes.to_crs(26986)
ped_nodes = ped_nodes.to_crs(26986)

In [4]:
#buffer!
tazs['geometry']=tazs.buffer(100)


In [5]:
#filter bike nodes so only get where multiple streets cross
bike_intersections = bike_nodes[bike_nodes['street_cou']>1]
ped_intersections = ped_nodes[ped_nodes['street_cou']>1]

In [6]:
#get counts of points in each polygon (buffered tazs)
spjoin_bike = gpd.sjoin(tazs, bike_intersections)
spjoin_ped = gpd.sjoin(tazs, ped_intersections)

In [7]:
#get count of points per polygon
pperp_bike = spjoin_bike.groupby(['taz']).count().reset_index()
pperp_ped = spjoin_ped.groupby(['taz']).count().reset_index()

In [8]:
#get a count - but has to be a column that has been 100% filled in the original data
pperp_bike['intersection_count'] =  pperp_bike['OBJECTID']
taz_ints_bike = pperp_bike[['taz','intersection_count']]

pperp_ped['intersection_count'] =  pperp_ped['OBJECTID']
taz_ints_ped = pperp_ped[['taz','intersection_count']]

In [9]:
#get square miles
tazs['Area_SqMi'] =  tazs['Shape_Area']*0.00000038610

In [10]:
#get density
taz_area = tazs[['taz','Area_SqMi']]
taz_den_bike = taz_ints_bike.merge(taz_area, how = 'outer', on='taz')
taz_den_ped = taz_ints_ped.merge(taz_area, how = 'outer', on='taz')

In [11]:
taz_den_bike['intersection_density'] = taz_den_bike['intersection_count']/taz_den_bike['Area_SqMi']
taz_den_ped['intersection_density'] = taz_den_ped['intersection_count']/taz_den_ped['Area_SqMi']

In [12]:
taz_den_bike.to_csv('E:\Shared drives\TMD_TSA\Data\\Non_Motorized_Networks\Intersections_per_TAZ\\bike_intersection_density.csv')
taz_den_ped.to_csv('E:\Shared drives\TMD_TSA\Data\\Non_Motorized_Networks\Intersections_per_TAZ\ped_intersection_density.csv')

In [13]:
taz_ints_bike.to_csv('E:\Shared drives\TMD_TSA\Data\\Non_Motorized_Networks\Intersections_per_TAZ\\bike_intersections_per_buffered_taz.csv')
taz_ints_ped.to_csv('E:\Shared drives\TMD_TSA\Data\\Non_Motorized_Networks\Intersections_per_TAZ\ped_intersections_per_buffered_taz.csv')